In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [3]:
from sklearn import linear_model

In [4]:
"""'
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [5]:
df_train.head()

smiles  feat_001  feat_002  \
0  c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...         0         0   
1  C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...         1         0   
2  [nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...         1         0   
3  [nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...         1         0   
4     c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1         0         0   

   feat_003  feat_004  feat_005  feat_006  feat_007  feat_008  feat_009  ...   \
0         0         0         1         0         1         0         0  ...    
1         0         0         1         0         1         0         0  ...    
2         0         0         1         1         1         0         0  ...    
3         0         0         1         1         1         0         0  ...    
4         0         0         1         0         1         0         0  ...    

   feat_248  feat_249  feat_250  feat_251  feat_252  feat_253  feat_254  \
0         1         0         0         0         0         0         0   
1         1         0         0         1         0         0         0   
2         1         0         0         0         1         0         0   
3         1         0         0         0         1         0         0   
4         1         0         0         0         0         0         0   

   feat_255  feat_256   gap  
0         0         0  1.19  
1         0         0  1.60  
2         0         0  1.49  
3         0         0  1.36  
4         0         0  1.98  

[5 rows x 258 columns]

In [6]:
df_test.head()

Id                                             smiles  feat_001  feat_002  \
0   1  c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...         0         0   
1   2         [nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1         0         0   
2   3  [nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...         1         0   
3   4  [nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...         1         0   
4   5  c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...         0         0   

   feat_003  feat_004  feat_005  feat_006  feat_007  feat_008    ...     \
0         0         0         1         1         1         0    ...      
1         0         0         1         1         1         0    ...      
2         0         0         1         1         1         0    ...      
3         0         0         1         1         1         0    ...      
4         0         0         1         0         1         0    ...      

   feat_247  feat_248  feat_249  feat_250  feat_251  feat_252  feat_253  \
0         0         1         0         0         0         0         0   
1         0         1         0         0         0         0         0   
2         0         1         0         0         0         0         0   
3         0         1         0         0         0         0         0   
4         0         1         0         0         0         0         0   

   feat_254  feat_255  feat_256  
0         0         0         0  
1         0         0         0  
2         0         0         0  
3         0         0         0  
4         0         0         0  

[5 rows x 258 columns]

In [7]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [59]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

smiles  feat_001  feat_002  \
0  c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...         0         0   
1  C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...         1         0   
2  [nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...         1         0   
3  [nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...         1         0   
4     c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1         0         0   

   feat_003  feat_004  feat_005  feat_006  feat_007  feat_008  feat_009  \
0         0         0         1         0         1         0         0   
1         0         0         1         0         1         0         0   
2         0         0         1         1         1         0         0   
3         0         0         1         1         1         0         0   
4         0         0         1         0         1         0         0   

     ...     feat_247  feat_248  feat_249  feat_250  feat_251  feat_252  \
0    ...            0         1         0         0         0         0   
1    ...            0         1         0         0         1         0   
2    ...            0         1         0         0         0         1   
3    ...            0         1         0         0         0         1   
4    ...            0         1         0         0         0         0   

   feat_253  feat_254  feat_255  feat_256  
0         0         0         0         0  
1         0         0         0         0  
2         0         0         0         0  
3         0         0         0         0  
4         0         0         0         0  

[5 rows x 257 columns]

In [60]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
# smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
# df_all['smiles_len'] = pd.DataFrame(smiles_len)

countC = np.vstack(df_all.smiles.astype(str).apply(lambda x: x.count('c')))
df_all['countC'] = pd.DataFrame(countC)

df_all.head()

# def func():
#     for elem in df_all.smiles:
#         num = elem.count('c')
#         print num
# func()


smiles  feat_001  feat_002  \
0  c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...         0         0   
1  C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...         1         0   
2  [nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...         1         0   
3  [nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...         1         0   
4     c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1         0         0   

   feat_003  feat_004  feat_005  feat_006  feat_007  feat_008  feat_009  \
0         0         0         1         0         1         0         0   
1         0         0         1         0         1         0         0   
2         0         0         1         1         1         0         0   
3         0         0         1         1         1         0         0   
4         0         0         1         0         1         0         0   

    ...    feat_248  feat_249  feat_250  feat_251  feat_252  feat_253  \
0   ...           1         0         0         0         0         0   
1   ...           1         0         0         1         0         0   
2   ...           1         0         0         0         1         0   
3   ...           1         0         0         0         1         0   
4   ...           1         0         0         0         0         0   

   feat_254  feat_255  feat_256  countC  
0         0         0         0      19  
1         0         0         0      12  
2         0         0         0      18  
3         0         0         0      18  
4         0         0         0      20  

[5 rows x 258 columns]

In [85]:
print (Y_train)

[ 1.19  1.6   1.49 ...,  2.03  1.74  1.72]


In [61]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print ("Train features:", X_train.shape)
print ("Train gap:", Y_train.shape)
print ("Test features:", X_test.shape)

Train features: (1000000, 257)
Train gap: (1000000,)
Test features: (824230, 257)


In [1]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

NameError: name 'LinearRegression' is not defined

In [52]:
def scorefunc(predictions,truth):
    sum=0
    N=len(predictions)
    for i in range(len(predictions)):
        sum+=np.square(predictions[i]-truth[i])
    ans=np.sqrt(sum/N)
    return ans

In [65]:
def runPrediction():
    
    def cross_validate(x, y, num):
        length = len(x)/num

        start = 0
        end = length
        for i in range(num):
            testData = []
            testData.append([x[start:end], y[start:end]])
            trainData = []
            trainData.append([np.concatenate([x[0:start], x[end:]]), np.concatenate([y[0:start], y[end:]])])
            
            print ("errors are " , output_errors(testData, trainData))

            start = end
            end += length
            
    def output_errors(test_array, train_array):
        result = []

        clf.fit(train_array[0][0], train_array[0][1])

        # predict data
        lasso_pred=clf.predict(test_array[0][0])

        # add the least square error of this test data
        error = scorefunc(lasso_pred,test_array[0][1])
        result.append(error)

        return result
    
    alphaStep = 1
    alphaParam = 1
    l1_ratioParam = 0.5
    while(alphaParam < 1.5):
        clf = linear_model.Lasso(alpha=alphaParam)
        clf2 = linear_model.LassoLars(alpha=alphaParam)
        clf3= linear_model.Ridge(alpha=alphaParam)
        clf4 = linear_model.ElasticNet(alpha=alphaParam, l1_ratio=l1_ratioParam)
        print (alphaParam)
        cross_validate(X_train, Y_train, 5)
        alphaParam += alphaStep 
    

runPrediction()


1
errors are  [0.40754417042315538]
errors are  [0.40651037528018236]
errors are  [0.40771203617527163]
errors are  [0.40661618518260284]
errors are  [0.40749854073060504]


/Users/aman/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/aman/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [65]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)


(1000000,)


In [16]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [91]:
write_to_file("sample1.csv", LR_pred)
write_to_file("sample2.csv", RF_pred)


In [17]:

#print(clf.coef_)
#print(clf.intercept_)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [19]:
write_to_file("sample3.csv",Lasso_pred)

In [20]:
#print(Lasso_pred)

In [67]:
scorefunc(Lasso_pred,Y_train)

0.4598646501926914

In [70]:
scorefunc(LR_pred,Y_train)

0.49212644983727222

/Users/aman/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/aman/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


errors are  [0.31531954268224488, 0.31447780090182492, 0.31571675858423265, 0.31525443798129121, 0.31562590666281792]


In [ ]:
#